# Support Vector Clustering (SVC)

De forma a demonstrar a aplicabilidade do SVC para problemas com dados reais, é proposto a análise de dados de operação de um processo Kraft presente em uma indústria de celulose real. Os dados a serem analisados consistem em dados de monitoramento de uma caldeira utilizada para queima de um licor proveniente da extração de celulose da madeira. Propõe-se o uso do SVC para o agrupamento de dados da caldeira em estado limpo e de dados da caldeira com depósito de sólidos. Para isso dados de operação em ambas as situações são fornecidos. Pelo fato de já se saber qual o estado da caldeira em cada amostra de dado, o problema elaborado pode ser considerado um caso de aprendizado semi-supervisionado, onde os dados são agrupados em duas classes já conhecidas.

### Bibliotecas a serem utilizadas

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cvx
from tqdm import tqdm
import sklearn.datasets
import time
plt.style.use('seaborn-whitegrid')

### Algoritmo Support Vector Clustering (SVC)

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cvx
from tqdm import tqdm
plt.style.use('seaborn-whitegrid')

class SupportVectorClustering():

    def __init__(self):
        pass

    def dataset(self, xs, xs_labels):
        self.xs = xs # dataset
        self.xs_labels = xs_labels
        self.N = len(xs) # number de instâncias

    def parameters(self, p=0.1, q=1):
        self.q = q # parâmetro kernel width 
        self.p = p # fração de bounded support vectors (BSVs) 
        self.C = 1/(self.N*p) # constante de penalização (1/C >= 1)
    
    def kernel(self, x1, x2):
        return np.exp(-self.q*np.sum((x1-x2)**2, axis=-1)) # gaussian kernel 

    def kernel_matrix(self):
        self.km = np.zeros((self.N, self.N))
        for i in range(self.N):
            for j in range(self.N):
                self.km[i,j] = self.kernel(self.xs[i], self.xs[j])

    # método de otimização
    def find_beta(self): 
        beta = cvx.Variable(self.N) # vetor de N dimensões
        objective = cvx.Maximize(cvx.sum(beta) - cvx.quad_form(beta, self.km))  # função objetivo 
        constraints = [0 <= beta, beta <= self.C, cvx.sum(beta) == 1] # restrições 
        prob = cvx.Problem(objective, constraints) # definição do problema de otimização
        prob.solve() # solução do problema de otimização
        self.beta = beta.value # valores ótimos das variáveis beta 

    # cálculo do raio da hiperesfera
    def r_func(self, x):
        return self.kernel(x, x) - 2*np.sum([self.beta[i]*self.kernel(self.xs[i],x) for i in range(self.N)]) + self.beta.T@self.km@self.beta
        # python > 3.5 @ matrix multiplication

    # amostragem de segmentos entre dois pontos
    def sample_segment(self,x1,x2,r,n=10):
        adj = True
        for i in range(n):
            x = x1 + (x2-x1)*i/(n+1) 
            if self.r_func(x) > r:
                adj = False
                return adj
        return adj
    
    # definição da matriz de adjacência
    def cluster(self):
        print('Calculating adjacency matrix... \n')
        svs_tmp = np.array(self.beta < self.C)*np.array(self.beta > 10**-8) # svs: 0 < beta < C
        self.svs = np.where(svs_tmp == True)[0] # índice support vectors 
        bsvs_tmp = np.array(self.beta >= self.C) # bsvs: beta == C ??
        self.bsvs = np.where(bsvs_tmp == True)[0] # índice bounded support vectors
        self.r = np.mean([self.r_func(self.xs[i]) for i in self.svs[:5]]) # why 5??
        self.adj = np.zeros((self.N, self.N)) # matriz adjacência
        # checar adjacência entre pontos 
        for i in tqdm(range(self.N)):
            if i not in self.bsvs:
                for j in range(i, self.N):
                    if j not in self.bsvs:
                        self.adj[i,j] = self.adj[j,i] = self.sample_segment(self.xs[i],self.xs[j],self.r)
    
    # definição dos clusters
    def return_clusters(self):
        ids = list(range(self.N))
        self.clusters = {}
        num_clusters = 0
        while ids:
            num_clusters += 1
            self.clusters[num_clusters] = []
            curr_id = ids.pop(0)
            queue = [curr_id]
            while queue:
                cid = queue.pop(0)
                for i in ids:
                    if self.adj[i,cid]:
                        queue.append(i)
                        ids.remove(i)
                self.clusters[num_clusters].append(cid)
        print('\n')
        print(f'The number of clusters is {num_clusters}')
    
    # resultados diagnóstico
    def results(self):
        clusters_results = np.zeros((len(self.clusters.keys()), 2))
        for i in self.clusters.keys():
            normal = 0
            fault = 0
            for j in self.clusters[i]:
                if self.xs_labels[j] == 0:
                    normal += 1
                else:
                    fault += 1   
            clusters_results[i-1][0] = normal
            clusters_results[i-1][1] = fault
        print([0, 1])
        print(clusters_results)
        print(len(clusters_results))
    
    # centróides dos clusters
    def clusters_centroids(self):
        self.centroids = dict()
        for key, values in self.clusters.items(): 
            sum_cluster_data = np.zeros(len(self.xs[0]))
            for j in values:
                sum_cluster_data += self.xs[j]
            centroid = sum_cluster_data / len(values)
            self.centroids[key] = centroid
        print(self.centroids)

    # teste do modelo
    def test_clustering(self, xs_test, xs_labels_test):
        self.xs_test = xs_test
        self.xs_labels_test = xs_labels_test
        clusters_similarity = np.zeros((len(xs_test),len(self.centroids.keys())))
        for i, x in enumerate(xs_test):
            for key, centroid in self.centroids.items():
                clusters_similarity[i,key-1] = np.linalg.norm(x-centroid, 2)
        cluster_assignment = np.argmin(clusters_similarity, axis = 1)
        print(cluster_assignment)
        
        if len(self.centroids.keys()) == 2:
            false_alarm = 0
            warn_miss = 0
            for i in range(len(self.xs_labels_test)):
                if (cluster_assignment[i] != self.xs_labels_test[i]):
                    if ((cluster_assignment[i] == 1) and (self.xs_labels_test[i] == 0)):
                        false_alarm += 1
                    else:
                        warn_miss += 1
            print(f'False Alarm {false_alarm*100/len(self.xs_labels_test)}%')
            print(f'Missed Warning {warn_miss*100/len(self.xs_labels_test)}%')
            print(f'Right Diagnostic {(len(self.xs_labels_test) - false_alarm - warn_miss)*100/len(self.xs_labels_test)}%')
        else:
            error = 0
            for i in range(len(self.xs_labels_test)):
                if cluster_assignment[i] != self.xs_labels_test[i]:
                    error += 1
            print(f'Right Diagnostic {(len(self.xs_labels_test) - error)*100/len(self.xs_labels_test)}%')
            print(f'Wrong Diagnostic {(error)*100/len(self.xs_labels_test)}%')


    def show_plot(self):
        labels = np.zeros(self.xs.shape[0]) # number of samples
        for i in self.clusters.keys():
            for j in self.clusters[i]:
                labels[j] = int(i) # cluster label for each point
        
        from pandas import DataFrame
        from matplotlib import pyplot
        df = DataFrame(dict(x=self.xs[:,0], y=self.xs[:,1], label=labels)) # table
        colors ={1:'r',2:'b',3:'g',4:'c',5:'m',6:'y',7:'k',8:'b',9:'c'}
        fig, ax = pyplot.subplots() 
        grouped = df.groupby('label')
        for key, group in grouped:
            group.plot(ax=ax, kind='scatter', x='x', y='y', label=key, color=colors[key])
        plt.show()

### Main()

Importação dos dados

In [19]:
f = open('caldeira_dataset.csv')
data = f.read()
f.close()
# conversão dos dados para matriz
lines = data.split('\n')
num_rows = len(lines)
num_cols = len(lines[0].split(';'))
float_data = np.zeros((num_rows, num_cols-2))
labels = np.zeros(len(float_data))
for i, line in enumerate(lines):
    values = [float(value) for value in line.split(';')[1:num_cols-1]]
    float_data[i,:] = values
    labels[i] = float(line[-1])

Divisão dos dados

In [20]:
normal_data = float_data[:100]
fault_data = float_data[2882:2982]
training_data = np.concatenate((normal_data, fault_data), axis = 0)

# rótulos dos dados de treinamento
normal_labels = labels[:100]
fault_labels = labels[2882:2982]
training_labels = np.concatenate((normal_labels, fault_labels), axis = 0)

# dados de teste
test_normal_data = float_data[1440:1540]
test_fault_data = float_data[4322:4422]
test_data = np.concatenate((test_normal_data, test_fault_data), axis = 0)

# rótulos dos dados de teste
test_normal_labels = labels[1440:1540]
test_fault_labels = labels[4322:4422]
test_data_labels = np.concatenate((test_normal_labels, test_fault_labels), axis = 0)

Normalização dos dados

In [21]:
mean = training_data.mean(axis=0)
std = training_data.std(axis=0)
training_data -= mean
training_data /= std
test_data -= mean
test_data /= std

Algoritmo SVC

In [22]:
svc = SupportVectorClustering()
svc.dataset(training_data, training_labels) # database
svc.parameters(p=0.005, q=0.24) # parâmetros # p = 0.005 q = 0.24 funcionou
svc.kernel_matrix() # matriz kernel
svc.find_beta() # solução problema de otimização
svc.cluster() # matriz de adjacência
svc.return_clusters() # define clusters
svc.results()
svc.clusters_centroids()
svc.test_clustering(test_data, test_data_labels)

  0%|                                                  | 0/200 [00:00<?, ?it/s]

Calculating adjacency matrix... 



100%|████████████████████████████████████████| 200/200 [11:04<00:00,  3.32s/it]



The number of clusters is 2
[0, 1]
[[100.   0.]
 [  0. 100.]]
2
{1: array([ 0.87852056, -0.99983026,  0.8487985 ,  0.20856193, -0.3303804 ,
       -0.67189928, -0.5794404 ,  0.99165125,  0.99701967, -0.47205518,
       -0.92167641,  0.68981489,  0.99712042, -0.38669433, -1.        ,
        0.9818645 , -0.0177221 , -0.99490296]), 2: array([-0.87852056,  0.99983026, -0.8487985 , -0.20856193,  0.3303804 ,
        0.67189928,  0.5794404 , -0.99165125, -0.99701967,  0.47205518,
        0.92167641, -0.68981489, -0.99712042,  0.38669433,  1.        ,
       -0.9818645 ,  0.0177221 ,  0.99490296])}
[1 1 1 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1